<a href="https://colab.research.google.com/github/Ratnadeep2007/Twitter_Sentiment/blob/main/Twitter_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.models import Sequential
import string
import nltk
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
path_twitter = '/content/drive/MyDrive/Machine Learning/twitter_train.csv'

In [ ]:
df_twitter = pd.read_csv(path_twitter, encoding='latin-1')

In [ ]:
df_twitter.head()

,Username,Games,Sentiment,Tweet
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [ ]:
# Text Preprocessing
df_twitter.shape

(74682, 4)

In [ ]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74682 entries, 0 to 74681
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Username   74682 non-null  int64 
 1   Games      74682 non-null  object
 2   Sentiment  74682 non-null  object
 3   Tweet      73996 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.3+ MB


In [ ]:
# shuffling the data
df_twitter = df_twitter.sample(frac=1)

In [ ]:
df_twitter['Sentiment'].value_counts()

Negative      22542
Positive      20832
Neutral       18318
Irrelevant    12990
Name: Sentiment, dtype: int64

In [ ]:
Y = df_twitter['Sentiment'].map({'Positive':0,'Negative':1,'Neutral':2,'Irrelevant':3}).values

In [ ]:
def clean_doc(sample):
  # split the sample
  tokens = sample.split()
  # prepare the regular expression to remove the punctuations
  re_punc = re.compile('[%s]' % re.escape(string.punctuation))
  tokens = [re_punc.sub('',word) for word in tokens]
  # removing non alphabetic or numerical values
  tokens = [ token for token in tokens if token.isalpha()]
  # remove stop words
  stop_words = set(stopwords.words('english'))
  tokens = [word for word in tokens if word not in stop_words]
  # filter out the small tokens
  tokens = [word for word in tokens if len(word)>1]
  tokens = [word.lower() for word in tokens]
  return tokens

In [ ]:
def join_token(tokens):
  return " ".join(tokens)

In [ ]:
X_raw = df_twitter.loc[:,'Tweet']

In [ ]:
X_raw[0]

'im getting on borderlands and i will murder you all ,'

In [ ]:
tw = X_raw[0]

In [ ]:
clean_txt = clean_doc(tw)

In [ ]:
clean_txt

['coming', 'borders', 'kill']

In [ ]:
# we just have to pass the entire data to clean_doc and join_token
clean_tokens = [clean_doc(sample) if pd.notna(sample) else [] for sample in X_raw]

In [ ]:
# cleaned text
X_clean = [join_token(token) for token in clean_tokens]

In [ ]:
X_clean[0]

'im getting borderlands murder'

In [ ]:
# model building and Vectorization
tk = Tokenizer()

In [ ]:
# implement the vocabulary
tk.fit_on_texts(X_clean)

In [ ]:
# building the vocabulary
V = tk.word_index
print(len(V))

32337


In [ ]:
# Convert the sentences into sequences
tweet_seq = tk.texts_to_sequences(X_clean)

In [ ]:
max_len = max([len(v) for v in tweet_seq])

In [ ]:
# padding of the sequences
tweet_pad_seq = pad_sequences(tweet_seq, padding='post', maxlen=max_len)

In [ ]:
# building the model
model_tweet = Sequential()
model_tweet.add(Embedding(len(V)+1, 64, input_length=max_len))
model_tweet.add(Flatten())
model_tweet.add(Dense(128, activation='tanh'))
model_tweet.add(Dense(5, activation='softmax'))
model_tweet.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [ ]:
model_tweet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 130, 64)           2069632   
                                                                 
 flatten (Flatten)           (None, 8320)              0         
                                                                 
 dense (Dense)               (None, 128)               1065088   
                                                                 
 dense_1 (Dense)             (None, 5)                 645       
                                                                 
Total params: 3135365 (11.96 MB)
Trainable params: 3135365 (11.96 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# fitting the model
model_tweet.fit(tweet_pad_seq, Y, validation_split=0.2, epochs=20)

Epoch 1/20
1868/1868 [==============================] - 71s 38ms/step - loss: 0.6849 - val_loss: 1.7898
Epoch 2/20
1868/1868 [==============================] - 67s 36ms/step - loss: 0.2081 - val_loss: 2.2720
Epoch 3/20
1868/1868 [==============================] - 66s 36ms/step - loss: 0.1347 - val_loss: 2.7946
Epoch 4/20
1868/1868 [==============================] - 68s 36ms/step - loss: 0.1166 - val_loss: 2.9296
Epoch 5/20
1868/1868 [==============================] - 67s 36ms/step - loss: 0.1054 - val_loss: 3.3490
Epoch 6/20
1868/1868 [==============================] - 66s 35ms/step - loss: 0.0973 - val_loss: 3.4011
Epoch 7/20
1868/1868 [==============================] - 66s 36ms/step - loss: 0.0929 - val_loss: 3.4075
Epoch 8/20
1868/1868 [==============================] - 68s 36ms/step - loss: 0.0884 - val_loss: 3.4916
Epoch 9/20
1868/1868 [==============================] - 70s 38ms/step - loss: 0.0856 - val_loss: 3.5457
Epoch 10/20
1868/1868 [==============================] - 70s 38m